In [30]:
import Conf_Calculator as CC
import pandas as pd
import json
import string_sum as sr
def common_tags(df_l, df_r):
    '''
    Utility Function: 
    '''
    left_ = df_l.columns.to_list()
    rigth_ = df_r.columns.to_list()
    return list(set(left_).intersection(rigth_))


In [31]:
OCpR, Prices, Orders = CC.dataframes_splitter(path_to_the_file = r'./uploads/MyCompany.xls', prices_tables=['Street_Prices', 'List_Prices'], order_tables = ['OrderData'])


In [32]:
finalizedModel = CC.loop_thru_dataframes(OCpR)



In [33]:
finalizedModel = finalizedModel[ list(filter( lambda x : 'tag' not in x,  finalizedModel.columns)) ]


In [34]:
config_boundles = pd.DataFrame()


In [35]:
config_boundles['code'] = finalizedModel.index
config_boundles['concat'] = pd.Series(finalizedModel.fillna('').values.tolist()).map(lambda x: CC.ConfigurationBoundleString(x))
dataFrameOfOrders = Orders[0]

In [36]:
boundlesToMonthSquenceOrders = CC.getOrderDat_returnBoundleToOrder(dataFrameOfOrders)
conf_list_of_boundles_data = list(map(list,zip(config_boundles['concat'], config_boundles['code'].astype(str))))
boundlesToMonthSquenceOrders_data = list(map(list,zip(  boundlesToMonthSquenceOrders['boundles'], boundlesToMonthSquenceOrders['month'].astype(str)+ "_" + boundlesToMonthSquenceOrders['seqNumb'].astype(str)  )))
b = sr.loop_two_lists_rayon_codes(conf_list_of_boundles_data , boundlesToMonthSquenceOrders_data, 0.8) #TODO: decide how to use this value 


In [37]:
fictionary_matching_results = {}
for i in b:
    if i.orderseq in fictionary_matching_results.keys():
        if (i.score > fictionary_matching_results[i.orderseq][1]):
            fictionary_matching_results[i.orderseq] =  [i.boundle_code, i.score]
    else:
        fictionary_matching_results[i.orderseq] =  [i.boundle_code, i.score]

month = []
seq   = []
match = []
for match_ in fictionary_matching_results.keys():
    l_identity_order = match_.split('_') 
    month.append(int(l_identity_order[0]))
    seq.append(int(l_identity_order[1]))
    match.append(fictionary_matching_results[match_][0])
d = {'Month': month, 'OrdSeq': seq, 'match': match}
matched_df = pd.DataFrame(data=d)

dataFrameOfOrders['unit_Price_QTY'] = dataFrameOfOrders['unit_Price'] * dataFrameOfOrders['qty']

orders_totals = dataFrameOfOrders[['Month', 'OrdSeq', 'unit_Price_QTY']].groupby(['Month', 'OrdSeq']).sum().reset_index()

orders_match_prices = pd.merge( matched_df, orders_totals, on=['Month', 'OrdSeq'],how='inner')
orders_match_prices.rename(columns={'match':'code'}, inplace=True)
orders_match_prices['code'] = orders_match_prices['code'].astype(int)


OCpR, Prices, Orders = CC.dataframes_splitter()
Model_Tags = CC.loop_thru_dataframes(OCpR)
Model_Tags = Model_Tags[ list(filter( lambda x : 'tag' in x,  Model_Tags.columns))  ]
Model_Tags['code'] = Model_Tags.index
Model_Tags['code'] = Model_Tags['code'].astype(int)

analyzed_orders = pd.merge(orders_match_prices, Model_Tags, on=['code'])
data =   json.loads(analyzed_orders.to_json(orient="records") ) 

In [46]:
#generating all dataFrames
MyCompany=r'./uploads/MyCompany.xls'
Competitor=r'./uploads/Competitor.xls'
dfs_ = CC.ADOT_dfs(MyCompany, Competitor)
Competitor0, Competitor1, Comp0, Comp1, Comp0_grouped, Comp1_grouped  = dfs_[0], dfs_[1], dfs_[2], dfs_[3], dfs_[4], dfs_[5]

#selecting the columns 
cmmn_tags_list = list(filter(lambda x: not(('q_by_Lp' in x) or ('q_by_Sp' in x)) , common_tags(Comp0_grouped, Comp1_grouped)))
ocpr_df = pd.merge(Comp0_grouped, Comp1_grouped, on=cmmn_tags_list ,how='inner', suffixes=('_0', '_1'))     


#criteria for selections
MyCompany_SP_Advantage  = ocpr_df[ocpr_df['q_by_Sp_0'] < ocpr_df['q_by_Sp_1']]
Competitor_SP_Advantage = ocpr_df[ocpr_df['q_by_Sp_0'] > ocpr_df['q_by_Sp_1']]


In [47]:
MyCompany_SP_Advantage.head()

,CPU_level_tag,cache_limit_tag,vSAN_level_tag,N_years_service_tag,q_by_Lp_0,q_by_Sp_0,q_by_Lp_1,q_by_Sp_1
0,Gold,multiple,ELA,3,6218.333333,3150.333333,6210.166667,3151.618333
1,Gold,multiple,Enterprise,3,6250.333333,3161.933333,6244.166667,3173.498333
2,Gold,single,ELA,3,6191.333333,3136.833333,6182.166667,3137.618333
3,Gold,single,Enterprise,3,6223.333333,3148.433333,6216.166667,3159.498333
8,Silver,multiple,ELA,3,6210.333333,3149.133333,6216.166667,3152.958333


In [38]:
analyzed_orders.head()

,Month,OrdSeq,code,unit_Price_QTY,CPU_level_tag,cache_limit_tag,vSAN_level_tag,N_years_service_tag
0,1,5,34,5539.8081,Platinum,single,Enterprise,3
1,2,14,34,5419.3590,Platinum,single,Enterprise,3
2,3,35,34,5337.2475,Platinum,single,Enterprise,3
3,1,6,26,5277.4788,Platinum,single,Enterprise,3
4,4,17,26,5084.5275,Platinum,single,Enterprise,3


In [49]:
dfs_ = CC.ADOT_dfs(MyCompany, Competitor)

Competitor0, Competitor1, Comp0, Comp1, Comp0_grouped, Comp1_grouped  = dfs_[0], dfs_[1], dfs_[2], dfs_[3], dfs_[4], dfs_[5]

TO_dict = {'Threats' : [Competitor1, Comp0_grouped], 'Opportunities':[Competitor0, Comp1_grouped]}
Threats_and_Opportunities = {}
for key_ in TO_dict.keys():
    
    _comparison_items =  TO_dict[key_]
    comp_0, comp_1 = _comparison_items[0], _comparison_items[1]

    competition = pd.merge(
                    comp_0,
                    comp_1,
                on= list(filter(lambda x: 'tag' in x,  common_tags(Competitor0,Comp1_grouped))),
                how='inner', 
                suffixes=('_AdvSrch', '_CompAdv'))  

    criteria_advantage_LP = competition['q_by_Lp_AdvSrch'] < competition['q_by_Lp_CompAdv']
    criteria_advantage_SP = competition['q_by_Sp_AdvSrch'] < competition['q_by_Sp_CompAdv']
    
    Advantage_LP = competition[criteria_advantage_LP]
    Advantage_SP = competition[criteria_advantage_SP]

    Threats_and_Opportunities[key_] = [{'ListPrices':Advantage_LP},{'StreetPrices':Advantage_SP}]

In [50]:
Threats_and_Opportunities['Opportunities']

g': 'single',
    'vSAN_level_tag': 'Enterprise',
    'N_years_service_tag': 3,
    'code': 7,
    'q_by_Lp_AdvSrch': 6185.0,
    'q_by_Sp_AdvSrch': 3131.7,
    'q_by_Lp_CompAdv': 6216.1666666667,
    'q_by_Sp_CompAdv': 3159.4983333333},
   {'CPU_level_tag': 'Gold',
    'cache_limit_tag': 'single',
    'vSAN_level_tag': 'Enterprise',
    'N_years_service_tag': 3,
    'code': 8,
    'q_by_Lp_AdvSrch': 6187.0,
    'q_by_Sp_AdvSrch': 3133.7,
    'q_by_Lp_CompAdv': 6216.1666666667,
    'q_by_Sp_CompAdv': 3159.4983333333},
   {'CPU_level_tag': 'Gold',
    'cache_limit_tag': 'single',
    'vSAN_level_tag': 'Enterprise',
    'N_years_service_tag': 3,
    'code': 9,
    'q_by_Lp_AdvSrch': 6153.0,
    'q_by_Sp_AdvSrch': 3118.9,
    'q_by_Lp_CompAdv': 6216.1666666667,
    'q_by_Sp_CompAdv': 3159.4983333333},
   {'CPU_level_tag': 'Gold',
    'cache_limit_tag': 'single',
    'vSAN_level_tag': 'Enterprise',
    'N_years_service_tag': 3,
    'code': 10,
    'q_by_Lp_AdvSrch': 6155.0,
    'q_by_Sp_Ad